In [1]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter
import math
from scipy import stats

In [2]:
stocks = pd.read_csv('sp500.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2261348771213,
 'week52high': 152.05,
 'week52low': 69.6,
 'week52highSplitAdjustOnly': 146.27,
 'week52lowSplitAdjustOnly': 69.87,
 'week52change': 0.988859866139843,
 'sharesOutstanding': 17354475576,
 'float': 0,
 'avg10Volume': 90785968,
 'avg30Volume': 91481945,
 'day200MovingAvg': 128.1,
 'day50MovingAvg': 126.59,
 'employees': 147755,
 'ttmEPS': 3.83,
 'ttmDividendRate': 0.8418319530781752,
 'dividendYield': 0.006393787357769461,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-21',
 'nextEarningsDate': '2021-04-22',
 'peRatio': 36.37826721207324,
 'beta': 1.3412874978141784,
 'maxChangePercent': 51.70809339895707,
 'year5ChangePercent': 4.435342241960305,
 'year2ChangePercent': 1.7268089883039712,
 'year1ChangePercent': 1.037263676779998,
 'ytdChangePercent': 0.007746172692538487,
 'month6ChangePercent': 0.15321487737324974,
 'month3ChangePercent': -0.024138841772197616,
 'month1ChangePercent': 0.11717689660667369,
 'day30ChangePe

In [4]:
data['year1ChangePercent']

1.037263676779998

Build DataFrame using IEX API


In [6]:
def chunks(lst, n):
    # Yield successive n-sized chunks from lst.
    for i in range(0, len(lst), n):
        yield lst[i:i +n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [11]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    #print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'
                        ],
                        index = my_columns),
            ignore_index=True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,134.81,0.845105,N/A
1,AAL,20.13,0.92413,N/A
2,AAP,205.30,0.777517,N/A
3,AAPL,132.17,1.050089,N/A
4,ABBV,113.23,0.460294,N/A
...,...,...,...,...
500,YUM,120.99,0.477113,N/A
501,ZBH,179.87,0.615517,N/A
502,ZBRA,500.62,1.566318,N/A
503,ZION,53.77,0.983632,N/A


Remove low-momentum stocks. <br>
Identify the 50 highest-momentum stocks in the S&P 500.


In [12]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,66.45,4.422185,N/A
1,FCX,34.99,4.002034,N/A
2,GPS,34.52,3.382942,N/A
3,KSS,59.61,2.696565,N/A
4,TPR,45.60,2.372055,N/A
5,ALGN,624.40,2.30527,N/A
6,NWSA,28.09,2.288817,N/A
7,URI,315.94,2.271603,N/A
8,SIVB,532.90,2.213708,N/A
9,MGM,41.04,2.193409,N/A


In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio: ")

    try:
        value = float(portfolio_size)
    
    except ValueError:
        print("Please insert a number. \n Try again: ")
        portfolio_size = input("Enter the value of your portfolio: ")

portfolio_input()
print(portfolio_size)


In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

Build a realistic momentum strategy <br>
Differentiate high quality vs low quality momentum stocks <br>

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_strin in symbol_strings:
    